In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.window import Window
from pyspark.sql.types import DateType
from urllib.request import urlopen
import json

In [0]:
class FavoDataTables():
  
  def __init__(self, spark):
    self.spark = spark
    self.url_orders = "https://favo-data-test.s3.amazonaws.com/TestFolder/orders.json"
    self.url_products = "https://favo-data-test.s3.amazonaws.com/TestFolder/products.json"
    self.url_buyers = "https://favo-data-test.s3.amazonaws.com/TestFolder/buyers.json"
    self.url_sellers = "https://favo-data-test.s3.amazonaws.com/TestFolder/sellers.json"
    
  def getDatafromUrl(self, url):
    data = urlopen(url).read().decode('utf-8')
    #Fix for phone number coming with '.' in buyers file
    data = data.replace(".","") if url.split("/")[4] == "buyers.json" else data
    jsonData = [json.loads(line) for line in data.replace(" ","").split("\r\n") if line != '']
    rdd = self.spark.sparkContext.parallelize(jsonData)
    df = self.spark.read.json(rdd)

    return df
  
  def getOrdersTable(self):
    orders = self.getDatafromUrl(self.url_orders)
    orders = orders.withColumn("product", explode("product"))\
        .withColumn("sku",col("product.sku"))\
        .withColumn("qt",col("product.qt"))\
        .drop("product")
    
    return orders
  
  def getProductTable(self):
    return self.getDatafromUrl(self.url_products)

  def getBuyerTable(self):
    buyers = self.getDatafromUrl(self.url_buyers)\
              .withColumnRenamed('start', 'buyer_start')
    return buyers
  
  def getSellersTable(self):
    sellers = self.getDatafromUrl(self.url_sellers)\
              .withColumnRenamed('start', 'seller_start')
    return sellers
  
  def getAllTables(self):
    return self.getOrdersTable(), self.getProductTable(), self.getBuyerTable(), self.getSellersTable()

In [0]:
def getSolutionOne(spark):
  """Funtion that solve Question1 of test, with this function intance the FavoClass and get all tables from the four different Favo json files and concatenate all of them.
  :param: spark -> SparkSession.
  """
  favo_data = FavoDataTables(spark)
  orders, products, buyers, sellers = favo_data.getAllTables()
  full_table = orders.alias("a").join(sellers, orders.seller_id == sellers.seller_id)\
              .join(buyers.alias("b"), orders.buyer_id == buyers.buyer_id, how = 'left')\
              .join(products.alias("c"), orders.sku == products.sku)\
              .drop(orders.seller_id)\
              .drop(buyers.buyer_id)\
              .drop(products.sku)
  return full_table

getSolutionOne(spark)

Out[34]: DataFrame[buyer_id: string, date: bigint, order_id: string, sku: bigint, qt: bigint, phone: bigint, seller: string, seller_id: string, seller_start: bigint, buyer: string, phone: bigint, buyer_start: bigint, id: string, name: string, price: double]

In [0]:
def getSolutionTwo(spark):
  """This function use date function in order to formatted date and to get year and week of date column after that usa lag function in order to get the difference between  weeks in order to know the increase of sales.
  :param: spark -> SparkSession.
  """
  full_table = getSolutionOne(spark)

  temp_table = full_table.select('seller_id','sku','qt','price',
                                 from_unixtime('date').cast(DateType()).alias('date'),
                                 from_unixtime('seller_start').cast(DateType()).alias('seller_start'))\
                    .withColumn('order_price',col('qt')*col('price'))\
                    .withColumn('year', year('date'))\
                    .withColumn("week", weekofyear("date"))\
                    .groupBy('year','week')\
                    .agg(sum('order_price').alias('week_sales'))\
                    .orderBy(col('year'), col('week'))

  window = Window.partitionBy("year").orderBy("week")
  temp_table.withColumn("sales_increase_by_week", col("week_sales") - lag(col("week_sales")).over(window))\
            .show()

getSolutionTwo(spark)

+----+----+------------------+----------------------+
year|week| week_sales|sales_increase_by_week|
+----+----+------------------+----------------------+
2020| 24| 53.98| null|
2020| 26| 175.4| 121.42000000000002|
2020| 27| 49.11| -126.29|
2020| 28| 107.12| 58.010000000000005|
2020| 30| 155.13| 48.00999999999999|
2020| 31| 279.98| 124.85000000000002|
2020| 32| 166.57| -113.41000000000003|
2020| 35|276.79999999999995| 110.22999999999996|
2020| 36| 369.61| 92.81000000000006|
2020| 37| 277.11| -92.5|
2020| 38| 102.17| -174.94|
2020| 39| 44.69| -57.480000000000004|
2020| 40| 353.78| 309.09|
2020| 41| 241.76| -112.01999999999998|
2020| 42| 151.34| -90.41999999999999|
2020| 43| 186.18| 34.84|
2020| 44| 61.85| -124.33000000000001|
2020| 45| 401.4| 339.54999999999995|
2020| 46|367.71000000000004| -33.68999999999994|
2020| 47| 316.17| -51.54000000000002|
+----+----+------------------+----------------------+
only showing top 20 rows

In [0]:
def getSolutionThree(spark):
  """This function get at first the total sales of all dataset and base on that check the percentage of contribution in sales of the sellers.
  :param: spark -> SparkSession.
  """
  full_table = getSolutionOne(spark)
  total_sales = full_table.withColumn("total_price",col("price")*col("qt"))\
    .agg(sum("total_price")).collect()[0]
  print(f"Total Sales: {total_sales[0]}")

  full_table.withColumn("total_price_perc",round(100*(col("price")*col("qt"))/total_sales[0],2))\
    .groupBy("seller_id")\
    .agg(sum("total_price_perc").alias("total_price_perc"))\
    .orderBy(col("total_price_perc").desc())\
    .show()

getSolutionThree(spark)

Total Sales: 16550.32
+--------------------+------------------+
 seller_id| total_price_perc|
+--------------------+------------------+
A5550636-F826-45A...| 3.15|
702BCB9F-7FED-4D6...|3.0799999999999996|
3958BE52-8FB1-44F...|2.9799999999999995|
AC896BAB-A90A-486...| 2.63|
778020D1-A104-4BE...|2.5300000000000002|
77BB24AC-F341-4A6...| 2.34|
2BCF11D0-7A70-4FE...| 2.04|
E549C7F6-7198-482...| 2.02|
1C0F030A-A398-447...|1.9900000000000002|
1F76B101-C67F-442...| 1.98|
3CD814C3-E583-484...| 1.96|
9F4FB492-798F-48C...| 1.94|
7DE2A626-F9CF-4D3...| 1.88|
AD3F0BC3-DFDB-4CF...| 1.88|
213EC031-B901-460...|1.8499999999999999|
5AF7E897-17D9-4D8...|1.8399999999999999|
BCBECEB8-462C-479...|1.8300000000000003|
F6FB69B6-6659-4F8...| 1.75|
032C593C-ED37-497...| 1.7|
D1683BAA-B1BE-47D...| 1.67|
+--------------------+------------------+
only showing top 20 rows

In [0]:
def getSolutionFour(spark):
  """This function at first get all SKUs sold with an empty buyer_id in order. Once we have that we filter this SKUs along all the order history in order to get the buyer that bought more qty  of that SKU, It is more likely that those buyers who buy those products in history, will buy more, that is why we filter and obtain the buyers who have bought a greater quantity of those SKUs.
  :param: spark -> SparkSession.
  """
  full_table = getSolutionOne(spark)
  sku_list = [int(row.sku)for row in full_table.select(col('sku')).where(col('buyer_id').isNull()).collect()]
  print(f"Sku with empty buyer_id info: {sku_list}")
  filtered_table = full_table.where(col('sku').isin(sku_list))\
    .where(col('buyer_id').isNotNull())\
    .groupBy(col('buyer_id'),col('sku'))\
    .agg(
        countDistinct(col('order_id')).alias('times_buyed_prd'),
        sum(col('qt')).alias('qty_buyed_prd'))

  filtered_table.select('buyer_id','sku','qty_buyed_prd',
                        row_number().over(Window.partitionBy("sku").orderBy(col("qty_buyed_prd").desc())).alias("rowNum"))\
              .where(col('rowNum')==1).show()

getSolutionFour(spark)

Sku with empty buyer_id info: [22, 72, 71, 57, 24, 7, 67, 2, 90, 75, 70, 66]
+--------------------+---+-------------+------+
 buyer_id|sku|qty_buyed_prd|rowNum|
+--------------------+---+-------------+------+
5E11C797-C168-431...| 2| 9| 1|
A33B2330-5679-4A6...| 7| 10| 1|
43244AFE-E6BC-4E0...| 22| 4| 1|
B34E74E2-781E-45E...| 24| 8| 1|
61256C0E-A78F-4F8...| 57| 11| 1|
D5871074-F993-4A1...| 66| 6| 1|
A0CDCA91-FB96-469...| 67| 7| 1|
331E9165-166E-404...| 70| 7| 1|
BB2C199B-EC21-422...| 71| 9| 1|
183C397E-71DA-494...| 72| 9| 1|
82046FD5-073A-4BF...| 75| 10| 1|
B92C8327-8869-443...| 90| 16| 1|
+--------------------+---+-------------+------+